In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('../data/fuel_consumption.csv', parse_dates=['YEAR'])
# Change Type of fuel to name
df['FUEL'] = df['FUEL'].replace({'X': 'Regular gasoline', 'Z': 'Premium gasoline', 'D': 'Diesel', 'E': 'Ethanol (E85)', 'N': 'Natural Gas'})
# Extract last caracter of transmission as number of gears
# ie. 816 cars have continuous variable transmission and don't have a number of gears
df['GEARS'] = df['TRANSMISSION'].str.extract(r'(\d+)$', expand=False)
df['TRANSMISSION'] = df['TRANSMISSION'].str.replace(r'\d+$', '')
df['TRANSMISSION'] = df['TRANSMISSION'].replace({'A': 'Automatic', 'AM': 'Automated manual', 'AS': 'Automatic with select shift', 'AV': 'Continuously variable', 'M': 'Manual'})
# Rename FUEL CONSUMPTION to CITY (L/100 km)
df = df.drop(columns=['COMB (mpg)'], axis = 1)
df = df.rename(columns={'FUEL CONSUMPTION': 'CITY (L/100 km)'})
df['MAKE'] = df['MAKE'].str.capitalize()

# Uniformize vehicle class
df['VEHICLE CLASS'] = df['VEHICLE CLASS'].str.capitalize()
df.loc[df['VEHICLE CLASS'].str.contains('Pickup truck'), 'VEHICLE CLASS'] = 'Pickup truck'
df.loc[df['VEHICLE CLASS'].str.contains('Station wagon'), 'VEHICLE CLASS'] = 'Station wagon'
df.loc[df['VEHICLE CLASS'].str.contains('Suv'), 'VEHICLE CLASS'] = 'SUV'
df.loc[df['VEHICLE CLASS'].str.contains('Van'), 'VEHICLE CLASS'] = 'Van'

# rename YEAR, VEHICLE CLASS, MAKE, MODEL, ENGINE SIZE, CYLINDERS, TRANSMISSION, FUEL, CITY (L/100 km), HWY (L/100 km), COMB (L/100 km), CO2 EMISSIONS (g/km)
df = df.rename(columns={'YEAR': 'Release year', 'GEARS' : 'Gears', 'VEHICLE CLASS': 'Vehicle class', 'MAKE': 'Make', 'MODEL': 'Model', 'ENGINE SIZE': 'Engine size (L)', 'CYLINDERS': 'Cylinders', 'TRANSMISSION': 'Transmission', 'FUEL': 'Fuel', 'CITY (L/100 km)': 'City (L/100 km)', 'COMB (L/100 km)': 'Mixed consumption (L/100 km)', 'HWY (L/100 km)': 'Highway (L/100 km)', 'EMISSIONS': 'CO2 emissions (g/km)'})
df['Release year'] = df['Release year'].dt.year
# Target - Features
X = df[['Make', 'Release year', 'Vehicle class', 'Fuel', 'Transmission', 'Gears', 'Engine size (L)', 'Cylinders']]
Y = df[['CO2 emissions (g/km)', 'Mixed consumption (L/100 km)', 'City (L/100 km)', 'Highway (L/100 km)']]

/var/folders/7l/wwrkhf0515x8dl_scksshbc00000gn/T/ipykernel_24274/2577772231.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['TRANSMISSION'] = df['TRANSMISSION'].str.replace(r'\d+$', '')


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numerical = X.select_dtypes(include=['int64', 'float64']).columns.values.tolist()
categorical = X.select_dtypes(include=['object']).columns.values.tolist()

# Preprocessing

preprocessor = ColumnTransformer(
transformers = [
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical),
    ('numerical', StandardScaler(), numerical)
    ])

In [7]:
from sklearn.linear_model import Lasso
from tempfile import mkdtemp
cachedir = mkdtemp()

pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('lasso', Lasso(max_iter = 10000))]
        )

# Linear regression
param = {
    'lasso__alpha': np.geomspace(0.000001, 1, 10),
}

In [8]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipeline, param_grid = param, cv=5, verbose=1, n_jobs=-1, scoring = 'r2', refit = True)
grid.fit(X, Y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/mathisderenne/opt/anaconda3/envs/streamlit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1490338.8499294966, tolerance: 6285.037067451805
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/opt/anaconda3/envs/streamlit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 975775.0643792879, tolerance: 6285.037067451805
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/opt/anaconda3/envs/streamlit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2839200.4668207066, tolerance: 6595.5729758714715
  model = cd_fast.sparse_en

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Make',
                                                                          'Vehicle '
                                                                          'class',
                                                                          'Fuel',
                                                                          'Transmission',
                                                                          'Gears']),
                                                                        ('numerical',
                                                                         StandardScaler(),
                                                                         ['Release '
                                                                          'year',
                                                                          'Engine '
                                                                          'size '
                                                                          '(L)',
                                                                          'Cylinders'])])),
                                       ('lasso', Lasso(max_iter=10000))]),
             n_jobs=-1,
             param_grid={'lasso__alpha': array([1.00000000e-06, 4.64158883e-06, 2.15443469e-05, 1.00000000e-04,
       4.64158883e-04, 2.15443469e-03, 1.00000000e-02, 4.64158883e-02,
       2.15443469e-01, 1.00000000e+00])},
             scoring='r2', verbose=1)

In [12]:
print(f"""Fitting completed :
Best parameters are {grid.best_params_}
With r2 score of {round(grid.best_score_, 3)}
> Interpretation : the model is able to explain {round(grid.best_score_ * 100)}% of the variance in the data. 
""")

Fitting completed :
Best parameters are {'lasso__alpha': 2.1544346900318823e-05}
With r2 score of 0.776
> Interpretation : the model is able to explain 78% of the variance in the data. 



In [10]:
grid.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_lasso__alpha', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [11]:
# Saving model 
import joblib

# joblib.dump(grid.best_estimator_, 'prediction_model.pkl')